In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, solve_bvp
from scipy.sparse import diags
from scipy.sparse.linalg import cg

In [2]:
### Constants:
ni, ne = 1e18, 1e18   ## (/m^3)
qi = +1   ## ion charge
qe = -1   ## electron charge

def conductivity_plasma(ni, ne, qi, qe):
    """ Calculates total conductivity of plasma

    Args:
        ni (float32): ion density
        ne (float32): electron density
        qi (float32): ion charge
        qe (float32): electron charge
    """
    return ni * qi + ne * qe

def surface_charge_density(sigma, v):
    """ Gives surface charge density of plasma

    Args:
        sigma (float32): Condunctivity
        v (float32): velocity particle
    """
    return sigma * v



def gauss_law_3d(rho, dx, dy, dz, epsilon_0=8.85e-12):
    """
    Solve Gauss's law to find the electric field using scipy.

    Parameters:
        rho (ndarray): 3D array of charge density.
        dx, dy, dz (float): Grid spacings in x, y, z directions.
        epsilon_0 (float): Permittivity of free space.

    Returns:
        Ex, Ey, Ez (ndarray): 3D arrays of electric field components.
    """
    # Get grid dimensions
    nx, ny, nz = rho.shape

    # Compute the divergence of E (∇·E = rho / ε₀)
    divergence = rho / epsilon_0

    # Define grid Laplacian operator using sparse matrix (7-point stencil)
    N = nx * ny * nz
    diag = -6 * np.ones(N)
    off_diag = np.ones(N)

    diagonals = [diag,
                 off_diag[:-1], off_diag[:-1],
                 off_diag[:-ny], off_diag[:-ny],
                 off_diag[:-nx*ny], off_diag[:-nx*ny]]

    offsets = [0, 1, -1, nx, -nx, nx*ny, -nx*ny]
    laplacian = diags(diagonals, offsets, shape=(N, N), format='csr')

    # Solve Poisson's equation (∇²V = -∇·E)
    divergence_flat = divergence.ravel()  # Flatten 3D to 1D
    potential_flat, info = cg(laplacian, -divergence_flat, rtol=1e-6)

    if info != 0:
        raise RuntimeError(f"CG solver did not converge (info={info})")

    # Reshape solution back to 3D
    potential = potential_flat.reshape((nx, ny, nz))

    # Compute the electric field components (E = -∇V)
    Ex = -(np.roll(potential, -1, axis=0) - np.roll(potential, 1, axis=0)) / (2 * dx)
    Ey = -(np.roll(potential, -1, axis=1) - np.roll(potential, 1, axis=1)) / (2 * dy)
    Ez = -(np.roll(potential, -1, axis=2) - np.roll(potential, 1, axis=2)) / (2 * dz)

    return Ex, Ey, Ez



In [1]:
### This is not complete yet I will complete it later and update on github in few days.